In [1]:
import types
import math
from typing import Callable

import torch
import torch.nn as nn
import torch.nn.functional as F

import timm

In [2]:
dino = timm.create_model('vit_small_patch16_224_dino', pretrained=True)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name vit_small_patch16_224_dino to current vit_small_patch16_224.dino.
  model = create_fn(


In [3]:
patch_size = [16, 16]
hooks = [2, 5, 8, 11]
hook_patch = True
start_index = 1

In [4]:
pretrained = nn.Module()
pretrained.model = dino

In [5]:
activations = {}
def get_activation(name: str) -> Callable:
    def hook(model, inputs, outputs):
        activations[name] = outputs
    return hook

In [6]:
for i in range(len(hooks)):
    pretrained.model.blocks[hooks[i]].register_forward_hook(get_activation(f'{i}')) # Get shape of 2nd, 5th, 6th... Block of ViT
if hook_patch: pretrained.model.pos_drop.register_forward_hook(get_activation('4'))

In [7]:
dino.eval()
output = dino(torch.rand(5, 3, 224, 224))
output.shape

torch.Size([5, 384])

In [8]:
for i in range(len(hooks)):
    print(f"Block {hooks[i]} Shape:", list(activations[f'{i}'].shape))
if hook_patch:
    print(f"Dropout Block Shape:", list(activations['4'].shape))

Block 2 Shape: [5, 197, 384]
Block 5 Shape: [5, 197, 384]
Block 8 Shape: [5, 197, 384]
Block 11 Shape: [5, 197, 384]
Dropout Block Shape: [5, 197, 384]


In [9]:
list(activations['4'].shape)

[5, 197, 384]

In [10]:
# B = batch size
# N = number of tokens
# C = embedding dim

In [11]:
B = 5
N = 77
C = 768
start_index = 1

x = torch.rand(B, N, C) # [5, 77, 768]

# Every local patch token gains global context
if start_index == 2:
    readout = (x[:, 0] + x[:, 1]) / 2 # (CLS + DIST) / 2
else:
    readout = x[:, 0]                 # CLS

# readout: [B,             C]
# x      : [B, N - 2 or 1, C]
out = x[: , start_index:] + readout.unsqueeze(1)
out.shape

torch.Size([5, 76, 768])

In [12]:
class Readout(nn.Module):
    """
    Adds CLS and/or DIST Tokens to the patches
    """
    def __init__(self, start_index = 1):
        super().__init__()

        self.start_index = start_index
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # X [B, N, C]
        # Every local patch token gains global context
        if self.start_index == 2:
            readout = (x[:, 0] + x[:, 1]) / 2 # (CLS + DIST) / 2
        else:
            readout = x[:, 0]                 # CLS
        
        # readout: [B,             C]
        # x      : [B, N - 2 or 1, C]
        return x[: , self.start_index: ] + readout.unsqueeze(1)

readout = Readout(start_index=1)
readout(torch.rand(5, 77, 768)).shape

torch.Size([5, 76, 768])

In [13]:
class Transpose(nn.Module):
    """
    from [B, N, C] to [B, C, N]
    """
    def __init__(self, dim1: int, dim2: int):
        super().__init__()
        self.dim1 = dim1
        self.dim2 = dim2
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return x.transpose(self.dim1, self.dim2).contiguous()

transpose = Transpose(1, 2)
transpose(torch.rand(5, 77, 768)).shape

torch.Size([5, 768, 77])

In [14]:
# _resize_pos_embed

start_index = 1
pos_embed = torch.rand(5, 197, 384)
gs_h = gs_w = 24

posemb_tok = pos_embed[:, :start_index]                     # posemb_tok:  [B,          1 or 2, C]
posemb_grid = pos_embed[0, start_index: ]                   # posemb_grid: [N - 1 or 2, C]

gs_old = int(math.sqrt(len(posemb_grid)))                   # gs_old, 14 or 16 .....

posemb_grid = posemb_grid.reshape(1, gs_old, gs_old, -1)    # posemb_grid: [1, 14, 14, C]
posemb_grid = posemb_grid.permute(0, 3, 1, 2)               # posemb_grid: [1, C, 14, 14]
posemb_grid = F.interpolate(posemb_grid, size=(gs_h, gs_w), 
                            mode="bilinear", 
                            align_corners=False)            # posemb_grid: [1, C, new_gs_w, new_gs_w]
posemb_grid = posemb_grid.permute(0, 2, 3, 1)               # posemb_grid: [1, new_gs_w, new_gs_w, C]
posemb_grid = posemb_grid.reshape(1, gs_h * gs_w, -1)       # posemb_grid: [1, new_gs_w x new_gs_w, C]
posemb_grid = posemb_grid.expand(pos_embed.shape[0], -1, -1)# posemb_grid: [B, new_gs_w x new_gs_w, C]           added Batch dim back
posemb = torch.cat([posemb_tok, posemb_grid], dim = 1)      # posemb_grid: [B, new_gs_w x new_gs_w + 1 or 2, C]  + 1 or 2 CLS token in dim = 1
posemb.shape


torch.Size([5, 577, 384])

In [15]:
def _resize_pos_embed(self, posemb: torch.Tensor, gs_h: int, gs_w: int) -> torch.Tensor:
    posemb_tok = pos_embed[:, : self.start_index]                    # posemb_tok:  [B,          1 or 2, C]
    posemb_grid = pos_embed[0, self.start_index :]                   # posemb_grid: [N - 1 or 2, C]

    gs_old = int(math.sqrt(len(posemb_grid)))                   # gs_old, 14 or 16 .....

    posemb_grid = posemb_grid.reshape(1, gs_old, gs_old, -1)    # posemb_grid: [1, 14, 14, C]
    posemb_grid = posemb_grid.permute(0, 3, 1, 2)               # posemb_grid: [1, C, 14, 14]
    posemb_grid = F.interpolate(posemb_grid, size=(gs_h, gs_w), 
                                mode="bilinear", 
                                align_corners=False)            # posemb_grid: [1, C, new_gs_w, new_gs_w]
    
    posemb_grid = posemb_grid.permute(0, 2, 3, 1)               # posemb_grid: [1, new_gs_w, new_gs_w, C]
    posemb_grid = posemb_grid.reshape(1, gs_h * gs_w, -1)       # posemb_grid: [1, new_gs_w x new_gs_w, C]
    posemb_grid = posemb_grid.expand(pos_embed.shape[0], -1, -1)# posemb_grid: [B, new_gs_w x new_gs_w, C]    # FIX
    posemb = torch.cat([posemb_tok, posemb_grid], dim = 1)      # posemb_grid: [B, new_gs_w x new_gs_w + 1 or 2, C] added Batch dim back + 1 or 2 CLS token
    return posemb

dino.start_index = 1
out = _resize_pos_embed(self = dino, posemb = torch.rand(5, 196, 384), gs_h = 24, gs_w = 24)
out.shape

torch.Size([5, 577, 384])

In [16]:
dino.patch_size = patch_size

In [17]:
x = torch.rand(5, 3, 256, 256)
x = pretrained.model.patch_embed.proj(x) # x: [B, embedding dimension, H // 16   W // 16]
x = x.flatten(2)                         # x: [B, embedding dimension, H // 16 x W // 16]
x = x.transpose(1,2)                     # x: [B, H // 16 x W // 16, embedding dimension]
# x: [B, patch_dim, embedding dimension]
# x: [B, N,         C]

pos_embed = _resize_pos_embed(self = dino, posemb = dino.pos_embed, gs_h = dino.patch_size[0], gs_w = dino.patch_size[1])

# # Adding CLS Tokens
cls_tokens = pretrained.model.cls_token             # cls_tokens: [1, 1, embedding dimension]
cls_tokens = cls_tokens.expand(x.shape[0], -1, -1)  # cls_tokens: [B, 1, embedding dimension]

x = torch.cat([cls_tokens, x], dim=1)               # x:   [B, N + 1, C]

assert x.shape == pos_embed.shape, f"x shape: {x.shape}, pos_embed: {pos_embed.shape}"
x = x + pos_embed
x = pretrained.model.pos_drop(x)                # x:   [B, N + 1, C]
for blk in pretrained.model.blocks:
    x = blk(x)                                  # x:   [B, N + 1, C]
x = pretrained.model.norm(x)                    # x:   [B, N + 1, C]
x.shape

torch.Size([5, 257, 384])

In [18]:
def forward_flex(self, x: torch.Tensor) -> torch.Tensor:
    x = self.patch_embed.proj(x) # x: [B, embedding dimension, H // 16   W // 16]
    x = x.flatten(2)                         # x: [B, embedding dimension, H // 16 x W // 16]
    x = x.transpose(1,2)                     # x: [B, H // 16 x W // 16, embedding dimension]
    # x: [B, patch_dim, embedding dimension]
    # x: [B, N,         C]

    pos_embed = _resize_pos_embed(self = self, posemb = self.pos_embed, gs_h = self.patch_size[0], gs_w = self.patch_size[1])

    # # Adding CLS Tokens
    cls_tokens = self.cls_token             # cls_tokens: [1, 1, embedding dimension]
    cls_tokens = cls_tokens.expand(x.shape[0], -1, -1)  # cls_tokens: [B, 1, embedding dimension]

    x = torch.cat([cls_tokens, x], dim=1)               # x:   [B, N + 1, C]

    assert x.shape == pos_embed.shape, f"x shape: {x.shape}, pos_embed: {pos_embed.shape}"
    x = x + pos_embed
    x = self.pos_drop(x)                # x:   [B, N + 1, C]
    for blk in self.blocks:
        x = blk(x)                                  # x:   [B, N + 1, C]
    x = self.norm(x)                    # x:   [B, N + 1, C]
    return x

x = torch.rand(5, 3, 256, 256)
out = forward_flex(self = pretrained, x = x)
out.shape

AttributeError: 'Module' object has no attribute 'patch_embed'

In [ ]:
def forward_vit(pretained: nn.Module, x: torch.Tensor) -> torch.Tensor:
    _ = pretrained.model.forward_flex(x) # No need to store output because the dict `activations` gets updated during ReadOut
    return {k: pretrained.rearrange(v) for k, v in activations.items()}

types.MethodType(forward_flex, pretrained.model)

<bound method forward_flex of VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=Fa

In [ ]:
pretrained.model.forward_flex

AttributeError: 'VisionTransformer' object has no attribute 'forward_flex'

In [ ]:
import types

# Define a class
class MyClass:
    def __init__(self, name):
        self.name = name

# Define an external function
def mthofn(self, greeting):
    return f"{greeting}, {self.name}!"

# Create an instance
obj = MyClass("Irfan")

# Bind the method to the instance
obj.mthofn = types.MethodType(mthofn, obj)

# Now you can call it like it's part of the class
print(obj.mthofn("Hello"))  # Output: Hello, Irfan!

Hello, Irfan!


In [ ]:
cls_tokens.shape

torch.Size([5, 1, 384])

In [ ]:
x + pos_embed

tensor([[[ 0.4503,  0.4302,  0.8052,  ...,  0.0496,  0.8904,  0.2330],
         [ 0.3433,  0.8414,  0.8782,  ...,  0.5155,  0.8013,  0.3979],
         [ 0.6798,  0.3992,  0.2170,  ...,  0.1009,  0.7024,  0.7537],
         ...,
         [ 0.4785,  0.6327,  0.5711,  ...,  0.4095,  0.6738,  0.8038],
         [ 0.5147,  0.2015,  0.8321,  ...,  0.8987,  0.6239,  0.8667],
         [ 0.4319,  0.9816,  0.2437,  ...,  0.1126,  0.4149,  0.1734]],

        [[ 0.8214,  0.4665,  0.3797,  ...,  0.2673,  0.8773,  0.6985],
         [ 0.3349,  0.8387,  0.8505,  ...,  0.5792,  0.7972,  0.3806],
         [ 0.6775,  0.3974,  0.2742,  ...,  0.1611,  0.7047,  0.7239],
         ...,
         [ 0.5014,  0.6828,  0.5618,  ...,  0.5725,  0.6808,  0.8190],
         [ 0.5231,  0.1941,  0.8398,  ...,  0.9578,  0.5998,  0.8520],
         [ 0.4508,  0.9752,  0.2939,  ...,  0.1116,  0.4257,  0.1686]],

        [[ 0.9700,  0.6100,  0.1174,  ...,  0.3687,  0.5279,  0.3219],
         [ 0.3347,  0.8453,  0.8652,  ...,  0

In [ ]:
cls_tokens.shape

torch.Size([5, 1, 384])

In [ ]:
x.shape

torch.Size([5, 196, 384])

In [ ]:
pretrained.model.cls_token.expand(x.shape[0], -1, -1).shape

torch.Size([5, 1, 384])

In [ ]:
torch.cat([cls_tokens, x], dim=1).shape

torch.Size([5, 197, 384])

In [ ]:
q, w, e, k

(5, 3, 224, 224)